# CS224N Assignment 1: Exploring Word Vectors (25 Points)
### <font color='blue'> Due 4:30pm, Tue January 14th 2025</font>

Welcome to CS224N! 

Before you start, make sure you **read the README.md** in the same directory as this notebook for important setup information. You need to install some Python libraries before you can successfully do this assignment. A lot of code is provided in this notebook, and we highly encourage you to read and understand it as part of the learning :)

If you aren't super familiar with Python, Numpy, or Matplotlib, we recommend you check out the review session on Friday. The session will be recorded and the material will be made available on our [website](http://web.stanford.edu/class/cs224n/index.html#schedule). The CS231N Python/Numpy [tutorial](https://cs231n.github.io/python-numpy-tutorial/) is also a great resource.


**Assignment Notes:** Please make sure to save the notebook as you go along. Submission Instructions are located at the bottom of the notebook.

In [1]:
# All Import Statements Defined Here
# Note: Do not add to this list.
# ----------------

import sys
assert sys.version_info[0] == 3
assert sys.version_info[1] >= 8

from platform import python_version
assert int(python_version().split(".")[1]) >= 5, "Please upgrade your Python version following the instructions in \
    the README.md file found in the same directory as this notebook. Your Python version is " + python_version()

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]

from datasets import load_dataset
imdb_dataset = load_dataset("stanfordnlp/imdb", name="plain_text")

import re
import numpy as np
import random
import scipy as sp
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

START_TOKEN = '<START>'
END_TOKEN = '<END>'
NUM_SAMPLES = 150

np.random.seed(0)
random.seed(0)
# ----------------

In [2]:
print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


## Word Vectors

Word Vectors are often used as a fundamental component for downstream NLP tasks, e.g. question answering, text generation, translation, etc., so it is important to build some intuitions as to their strengths and weaknesses. Here, you will explore two types of word vectors: those derived from *co-occurrence matrices*, and those derived via *GloVe*. 

**Note on Terminology:** The terms "word vectors" and "word embeddings" are often used interchangeably. The term "embedding" refers to the fact that we are encoding aspects of a word's meaning in a lower dimensional space. As [Wikipedia](https://en.wikipedia.org/wiki/Word_embedding) states, "*conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with a much lower dimension*".

## Part 1: Count-Based Word Vectors (10 points)

Most word vector models start from the following idea:

*You shall know a word by the company it keeps ([Firth, J. R. 1957:11](https://en.wikipedia.org/wiki/John_Rupert_Firth))*

Many word vector implementations are driven by the idea that similar words, i.e., (near) synonyms, will be used in similar contexts. As a result, similar words will often be spoken or written along with a shared subset of words, i.e., contexts. By examining these contexts, we can try to develop embeddings for our words. With this intuition in mind, many "old school" approaches to constructing word vectors relied on word counts. Here we elaborate upon one of those strategies, *co-occurrence matrices* (for more information, see [here](https://web.stanford.edu/~jurafsky/slp3/6.pdf) or [here](https://web.archive.org/web/20190530091127/https://medium.com/data-science-group-iitr/word-embedding-2d05d270b285)).

### Co-Occurrence

A co-occurrence matrix counts how often things co-occur in some environment. Given some word $w_i$ occurring in the document, we consider the *context window* surrounding $w_i$. Supposing our fixed window size is $n$, then this is the $n$ preceding and $n$ subsequent words in that document, i.e. words $w_{i-n} \dots w_{i-1}$ and $w_{i+1} \dots w_{i+n}$. We build a *co-occurrence matrix* $M$, which is a symmetric word-by-word matrix in which $M_{ij}$ is the number of times $w_j$ appears inside $w_i$'s window among all documents.

**Example: Co-Occurrence with Fixed Window of n=1**:

Document 1: "all that glitters is not gold"

Document 2: "all is well that ends well"


|     *    | `<START>` | all | that | glitters | is   | not  | gold  | well | ends | `<END>` |
|----------|-------|-----|------|----------|------|------|-------|------|------|-----|
| `<START>`    | 0     | 2   | 0    | 0        | 0    | 0    | 0     | 0    | 0    | 0   |
| all      | 2     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| that     | 0     | 1   | 0    | 1        | 0    | 0    | 0     | 1    | 1    | 0   |
| glitters | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| is       | 0     | 1   | 0    | 1        | 0    | 1    | 0     | 1    | 0    | 0   |
| not      | 0     | 0   | 0    | 0        | 1    | 0    | 1     | 0    | 0    | 0   |
| gold     | 0     | 0   | 0    | 0        | 0    | 1    | 0     | 0    | 0    | 1   |
| well     | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 1    | 1   |
| ends     | 0     | 0   | 1    | 0        | 0    | 0    | 0     | 1    | 0    | 0   |
| `<END>`      | 0     | 0   | 0    | 0        | 0    | 0    | 1     | 1    | 0    | 0   |

In NLP, we commonly use `<START>` and `<END>` tokens to mark the beginning and end of sentences, paragraphs, or documents. These tokens are included in co-occurrence counts, encapsulating each document, for example: "`<START>` All that glitters is not gold `<END>`".

The matrix rows (or columns) provide word vectors based on word-word co-occurrence, but they can be large. To reduce dimensionality, we employ Singular Value Decomposition (SVD), akin to PCA, selecting the top $k$ principal components. The SVD process decomposes the co-occurrence matrix $A$ into singular values in the diagonal $S$ matrix and new, shorter word vectors in $U_k$.

This dimensionality reduction maintains semantic relationships; for instance, *doctor* and *hospital* will be closer than *doctor* and *dog*.

For those unfamiliar with eigenvalues and SVD, a beginner-friendly introduction to SVD is available [here](https://davetang.org/file/Singular_Value_Decomposition_Tutorial.pdf). Additional resources for in-depth understanding include lectures [7](https://web.stanford.edu/class/cs168/l/l7.pdf), [8](http://theory.stanford.edu/~tim/s15/l/l8.pdf), and [9](https://web.stanford.edu/class/cs168/l/l9.pdf) of CS168, providing high-level treatment of these algorithms. For practical implementation, utilizing pre-programmed functions from Python packages like numpy, scipy, or sklearn is recommended. While applying full SVD to large corpora can be memory-intensive, scalable techniques such as Truncated SVD exist for extracting the top $k$ vector components efficiently.

### Plotting Co-Occurrence Word Embeddings

Here, we will be using the Large Movie Review Dataset. This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well. We provide a `read_corpus` function below that pulls out the text of a movie review from the dataset. The function also adds `<START>` and `<END>` tokens to each of the documents, and lowercases words. You do **not** have to perform any other kind of pre-processing.

In [3]:
def read_corpus():
    """ Read files from the Large Movie Review Dataset.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """
    files = imdb_dataset["train"]["text"][:NUM_SAMPLES]
    return [[START_TOKEN] + [re.sub(r'[^\w]', '', w.lower()) for w in f.split(" ")] + [END_TOKEN] for f in files]

Let's have a look what these documents are like….

In [4]:
imdb_corpus = read_corpus()
pprint.pprint(imdb_corpus[:3], compact=True, width=100)
print("corpus size: ", len(imdb_corpus[0]))

[['<START>', 'i', 'rented', 'i', 'am', 'curiousyellow', 'from', 'my', 'video', 'store', 'because',
  'of', 'all', 'the', 'controversy', 'that', 'surrounded', 'it', 'when', 'it', 'was', 'first',
  'released', 'in', '1967', 'i', 'also', 'heard', 'that', 'at', 'first', 'it', 'was', 'seized',
  'by', 'us', 'customs', 'if', 'it', 'ever', 'tried', 'to', 'enter', 'this', 'country', 'therefore',
  'being', 'a', 'fan', 'of', 'films', 'considered', 'controversial', 'i', 'really', 'had', 'to',
  'see', 'this', 'for', 'myselfbr', 'br', 'the', 'plot', 'is', 'centered', 'around', 'a', 'young',
  'swedish', 'drama', 'student', 'named', 'lena', 'who', 'wants', 'to', 'learn', 'everything',
  'she', 'can', 'about', 'life', 'in', 'particular', 'she', 'wants', 'to', 'focus', 'her',
  'attentions', 'to', 'making', 'some', 'sort', 'of', 'documentary', 'on', 'what', 'the', 'average',
  'swede', 'thought', 'about', 'certain', 'political', 'issues', 'such', 'as', 'the', 'vietnam',
  'war', 'and', 'race', 'issu

### Question 1.1: Implement `distinct_words` [code] (2 points)

Write a method to work out the distinct words (word types) that occur in the corpus.

You can use `for` loops to process the input `corpus` (a list of list of strings), but try using Python list comprehensions (which are generally faster). In particular, [this](https://coderwall.com/p/rcmaea/flatten-a-list-of-lists-in-one-line-in-python) may be useful to flatten a list of lists. If you're not familiar with Python list comprehensions in general, here's [more information](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Comprehensions.html).

Your returned `corpus_words` should be sorted. You can use python's `sorted` function for this.

You may find it useful to use [Python sets](https://www.w3schools.com/python/python_sets.asp) to remove duplicate words.

In [8]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): sorted list of distinct words across the corpus
            n_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    n_corpus_words = -1
    
    # ------------------
    # Write your implementation here.    
   
    corpus_words = sorted(list(set([word for docs in corpus for word in docs]))) 
    n_corpus_words = len(corpus_words)
    
    # ------------------

    return corpus_words, n_corpus_words

In [10]:
# ---------------------
# Run this sanity check
# Note that this not an exhaustive check for correctness.
# ---------------------

# Define toy corpus
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
test_corpus_words, num_corpus_words = distinct_words(test_corpus)

# Correct answers
ans_test_corpus_words = sorted([START_TOKEN, "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", END_TOKEN])
ans_num_corpus_words = len(ans_test_corpus_words)

# Test correct number of words
assert(num_corpus_words == ans_num_corpus_words), "Incorrect number of distinct words. Correct: {}. Yours: {}".format(ans_num_corpus_words, num_corpus_words)

# Test correct words
assert (test_corpus_words == ans_test_corpus_words), "Incorrect corpus_words.\nCorrect: {}\nYours:   {}".format(str(ans_test_corpus_words), str(test_corpus_words))


print (test_corpus_words)
print (num_corpus_words)
# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

['<END>', '<START>', 'All', "All's", 'ends', 'glitters', 'gold', "isn't", 'that', 'well']
10
--------------------------------------------------------------------------------
Passed All Tests!
--------------------------------------------------------------------------------


### Question 1.2: Implement `compute_co_occurrence_matrix` [code] (3 points)

Write a method that constructs a co-occurrence matrix for a certain window-size $n$ (with a default of 4), considering words $n$ before and $n$ after the word in the center of the window. Here, we start to use `numpy (np)` to represent vectors, matrices, and tensors. If you're not familiar with NumPy, there's a NumPy tutorial in the second half of this cs231n [Python NumPy tutorial](http://cs231n.github.io/python-numpy-tutorial/).


In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).
    
        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.
              
              For example, if we take the document "<START> All that glitters is not gold <END>" with window size of 4,
              "All" will co-occur with "<START>", "that", "glitters", "is", and "not".
    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (a symmetric numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): 
                Co-occurence matrix of word counts. 
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, n_words = distinct_words(corpus)
    M = None
    word2ind = {}
    
    # ------------------
    # Create word to index mapping
    word2ind = {word: i for i, word in enumerate(words)}
    
    # Initialize co-occurrence matrix with zeros
    M = np.zeros((n_words, n_words))
    
    # Iterate through each document
    for document in corpus:
        # For each word position in the document (as center word)
        for center_idx in range(len(document)):
            center_word = document[center_idx]
            center_word_ind = word2ind[center_word]
            
            # Look at words within the window
            # Window goes from center_idx - window_size to center_idx + window_size
            start_idx = max(0, center_idx - window_size)
            end_idx = min(len(document), center_idx + window_size + 1)
            
            # Count co-occurrences
            for context_idx in range(start_idx, end_idx):
                if context_idx != center_idx:  # Don't count the word with itself
                    context_word = document[context_idx]
                    context_word_ind = word2ind[context_word]
                    M[center_word_ind, context_word_ind] += 1
    
    # ------------------

    return M, word2ind

In [ ]:
# ---------------------
# Run this sanity check
# Note that this is not an exhaustive check for correctness.
# ---------------------

# Define toy corpus and get student's co-occurrence matrix
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
M_test, word2ind_test = compute_co_occurrence_matrix(test_corpus, window_size=1)

# Correct M and word2ind
M_test_ans = np.array( 
    [[0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,],
     [0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,],
     [0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,],
     [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,],
     [0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,],
     [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,],
     [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,],
     [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,],
     [0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,],
     [1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,]]
)
ans_test_corpus_words = sorted([START_TOKEN, "All", "ends", "that", "gold", "All's", "glitters", "isn't", "well", END_TOKEN])
word2ind_ans = dict(zip(ans_test_corpus_words, range(len(ans_test_corpus_words))))

# Test correct word2ind
assert (word2ind_ans == word2ind_test), "Your word2ind is incorrect:\nCorrect: {}\nYours: {}".format(word2ind_ans, word2ind_test)

# Test correct M shape
assert (M_test.shape == M_test_ans.shape), "M matrix has incorrect shape.\nCorrect: {}\nYours: {}".format(M_test.shape, M_test_ans.shape)

# Test correct M values
for w1 in word2ind_ans.keys():
    idx1 = word2ind_ans[w1]
    for w2 in word2ind_ans.keys():
        idx2 = word2ind_ans[w2]
        student = M_test[idx1, idx2]
        correct = M_test_ans[idx1, idx2]
        if student != correct:
            print("Correct M:")
            print(M_test_ans)
            print("Your M: ")
            print(M_test)
            raise AssertionError("Incorrect count at index ({}, {})=({}, {}) in matrix M. Yours has {} but should have {}.".format(idx1, idx2, w1, w2, student, correct))

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

### Question 1.3: Implement `reduce_to_k_dim` [code] (1 point)

Construct a method that performs dimensionality reduction on the matrix to produce k-dimensional embeddings. Use SVD to take the top k components and produce a new matrix of k-dimensional embeddings. 

**Note:** All of numpy, scipy, and scikit-learn (`sklearn`) provide *some* implementation of SVD, but only scipy and sklearn provide an implementation of Truncated SVD, and only sklearn provides an efficient randomized algorithm for calculating large-scale Truncated SVD. So please use [sklearn.decomposition.TruncatedSVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

In [ ]:
def reduce_to_k_dim(M, k=2):
    """ Reduce a co-occurence count matrix of dimensionality (num_corpus_words, num_corpus_words)
        to a matrix of dimensionality (num_corpus_words, k) using the following SVD function from Scikit-Learn:
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
    
        Params:
            M (numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): co-occurence matrix of word counts
            k (int): embedding size of each word after dimension reduction
        Return:
            M_reduced (numpy matrix of shape (number of corpus words, k)): matrix of k-dimensioal word embeddings.
                    In terms of the SVD from math class, this actually returns U * S
    """    
    n_iters = 10    # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))
    
    # ------------------
    # Write your implementation here.
    svd = TruncatedSVD(n_components=k, n_iter=n_iters, random_state=0)
    M_reduced = svd.fit_transform(M)
    
    # ------------------

    print("Done.")
    return M_reduced

In [ ]:
# ---------------------
# Run this sanity check
# Note that this is not an exhaustive check for correctness 
# In fact we only check that your M_reduced has the right dimensions.
# ---------------------

# Define toy corpus and run student code
test_corpus = ["{} All that glitters isn't gold {}".format(START_TOKEN, END_TOKEN).split(" "), "{} All's well that ends well {}".format(START_TOKEN, END_TOKEN).split(" ")]
M_test, word2ind_test = compute_co_occurrence_matrix(test_corpus, window_size=1)
M_test_reduced = reduce_to_k_dim(M_test, k=2)

# Test proper dimensions
assert (M_test_reduced.shape[0] == 10), "M_reduced has {} rows; should have {}".format(M_test_reduced.shape[0], 10)
assert (M_test_reduced.shape[1] == 2), "M_reduced has {} columns; should have {}".format(M_test_reduced.shape[1], 2)

# Print Success
print ("-" * 80)
print("Passed All Tests!")
print ("-" * 80)

### Question 1.4: Implement `plot_embeddings` [code] (1 point)

Here you will write a function to plot a set of 2D vectors in 2D space. For graphs, we will use Matplotlib (`plt`).

For this example, you may find it useful to adapt [this code](http://web.archive.org/web/20190924160434/https://www.pythonmembers.club/2018/05/08/matplotlib-scatter-plot-annotate-set-text-at-label-each-point/). In the future, a good way to make a plot is to look at [the Matplotlib gallery](https://matplotlib.org/gallery/index.html), find a plot that looks somewhat like what you want, and adapt the code they give.

In [ ]:
def plot_embeddings(M_reduced, word2ind, words):
    """ Plot in a scatterplot the embeddings of the words specified in the list "words".
        NOTE: do not plot all the words listed in M_reduced / word2ind.
        Include a label next to each point.
        
        Params:
            M_reduced (numpy matrix of shape (number of unique words in the corpus , 2)): matrix of 2-dimensioal word embeddings
            word2ind (dict): dictionary that maps word to indices for matrix M
            words (list of strings): words whose embeddings we want to visualize
    """

    # ------------------
    # Write your implementation here.
    # Extract x and y coordinates for the specified words
    x_coords = [M_reduced[word2ind[word]][0] for word in words]
    y_coords = [M_reduced[word2ind[word]][1] for word in words]

        # ------------------

    # Create scatter plot    

    plt.figure(figsize=(10, 5))    plt.show()

    plt.scatter(x_coords, y_coords, marker='o')    plt.grid(True, alpha=0.3)

        plt.title('Word Embeddings Visualization')

    # Add labels for each point    plt.ylabel('Dimension 2')

    for i, word in enumerate(words):    plt.xlabel('Dimension 1')

        plt.annotate(word, xy=(x_coords[i], y_coords[i]),     

                    xytext=(5, 2), textcoords='offset points',                    ha='right', fontsize=10)

In [ ]:
# ---------------------
# Run this sanity check
# Note that this is not an exhaustive check for correctness.
# The plot produced should look like the included file question_1.4_test.png 
# ---------------------

print ("-" * 80)
print ("Outputted Plot:")

M_reduced_plot_test = np.array([[1, 1], [-1, -1], [1, -1], [-1, 1], [0, 0]])
word2ind_plot_test = {'test1': 0, 'test2': 1, 'test3': 2, 'test4': 3, 'test5': 4}
words = ['test1', 'test2', 'test3', 'test4', 'test5']
plot_embeddings(M_reduced_plot_test, word2ind_plot_test, words)

print ("-" * 80)

### Question 1.5: Co-Occurrence Plot Analysis [written] (3 points)

Now we will put together all the parts you have written! We will compute the co-occurrence matrix with fixed window of 4 (the default window size), over the Large Movie Review corpus. Then we will use TruncatedSVD to compute 2-dimensional embeddings of each word. TruncatedSVD returns U\*S, so we need to normalize the returned vectors, so that all the vectors will appear around the unit circle (therefore closeness is directional closeness). **Note**: The line of code below that does the normalizing uses the NumPy concept of *broadcasting*. If you don't know about broadcasting, check out
[Computation on Arrays: Broadcasting by Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html).

Run the below cell to produce the plot. It can take up to a few minutes to run.

In [ ]:
# -----------------------------
# Run This Cell to Produce Your Plot
# ------------------------------
imdb_corpus = read_corpus()
M_co_occurrence, word2ind_co_occurrence = compute_co_occurrence_matrix(imdb_corpus)
M_reduced_co_occurrence = reduce_to_k_dim(M_co_occurrence, k=2)

# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced_co_occurrence, axis=1)
M_normalized = M_reduced_co_occurrence / M_lengths[:, np.newaxis] # broadcasting

words = ['movie', 'book', 'mysterious', 'story', 'fascinating', 'good', 'interesting', 'large', 'massive', 'huge']

plot_embeddings(M_normalized, word2ind_co_occurrence, words)

**Verify that your figure matches "question_1.5.png" in the assignment zip. If not, use the figure in "question_1.5.png" to answer the next two questions.**

a. Find at least two groups of words that cluster together in 2-dimensional embedding space. Give an explanation for each cluster you observe.

#### <font color="red">Write your answer here.</font>


**Cluster 1: Size-related words** - The words 'large', 'massive', and 'huge' cluster together because they all describe physical size and magnitude. These words frequently appear in similar contexts when describing objects, spaces, or quantities in movie reviews.

**Cluster 3: Positive descriptive words** - The words 'good', 'interesting', and 'fascinating' may cluster together as they are all positive adjectives commonly used to describe movies, performances, or plot elements in reviews.

**Cluster 2: Media/narrative words** - The words 'movie', 'book', and 'story' cluster together because they all refer to narrative media or storytelling formats. In movie reviews, these words often co-occur when discussing plot, adaptations, or comparing different media formats.

b. What doesn't cluster together that you might think should have? Describe at least two examples.

#### <font color="red">Write your answer here.</font>



**Example 1:** 'Good' and 'fascinating' might not cluster as closely as expected, even though both are positive descriptive words. This could be because 'good' is a more general, frequently-used term that appears in many different contexts, while 'fascinating' is more specific and sophisticated, appearing in more limited contexts.**Example 2:** 'Mysterious' might not cluster with 'interesting' or 'fascinating', even though all three can describe engaging content. 'Mysterious' has a more specific connotation related to suspense and intrigue, so it may co-occur with a different set of words (like 'dark', 'thriller', 'suspense') compared to the more general positive descriptors.


## Part 2: Prediction-Based Word Vectors (15 points)

As discussed in class, more recently prediction-based word vectors have demonstrated better performance, such as word2vec and GloVe (which also utilizes the benefit of counts). Here, we shall explore the embeddings produced by GloVe. Please revisit the class notes and lecture slides for more details on the word2vec and GloVe algorithms. If you're feeling adventurous, challenge yourself and try reading [GloVe's original paper](https://nlp.stanford.edu/pubs/glove.pdf).

Then run the following cells to load the GloVe vectors into memory. **Note**: If this is your first time to run these cells, i.e. download the embedding model, it will take a couple minutes to run. If you've run these cells before, rerunning them will load the model without redownloading it, which will take about 1 to 2 minutes.

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each length 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()

#### Note: If you are receiving a "reset by peer" error, rerun the cell to restart the download. 

### Reducing dimensionality of Word Embeddings
Let's directly compare the GloVe embeddings to those of the co-occurrence matrix. In order to avoid running out of memory, we will work with a sample of 40000 GloVe vectors instead.
Run the following cells to:

1. Put 40000 Glove vectors into a matrix M
2. Run `reduce_to_k_dim` (your Truncated SVD function) to reduce the vectors from 200-dimensional to 2-dimensional.

In [ ]:
def get_matrix_of_vectors(wv_from_bin, required_words):
    """ Put the GloVe vectors into a matrix M.
        Param:
            wv_from_bin: KeyedVectors object; the 400000 GloVe vectors loaded from file
        Return:
            M: numpy matrix shape (num words, 200) containing the vectors
            word2ind: dictionary mapping each word to its row number in M
    """
    import random
    words = list(wv_from_bin.index_to_key)
    print("Shuffling words ...")
    random.seed(225)
    random.shuffle(words)
    print("Putting %i words into word2ind and matrix M..." % len(words))
    word2ind = {}
    M = []
    curInd = 0
    for w in words:
        try:
            M.append(wv_from_bin.get_vector(w))
            word2ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    for w in required_words:
        if w in words:
            continue
        try:
            M.append(wv_from_bin.get_vector(w))
            word2ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    M = np.stack(M)
    print("Done.")
    return M, word2ind

In [ ]:
# -----------------------------------------------------------------
# Run Cell to Reduce 200-Dimensional Word Embeddings to k Dimensions
# Note: This should be quick to run
# -----------------------------------------------------------------
M, word2ind = get_matrix_of_vectors(wv_from_bin, words)
M_reduced = reduce_to_k_dim(M, k=2)

# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced, axis=1)
M_reduced_normalized = M_reduced / M_lengths[:, np.newaxis] # broadcasting

**Note: If you are receiving out of memory issues on your local machine, try closing other applications to free more memory on your device. You may want to try restarting your machine so that you can free up extra memory. Then immediately run the jupyter notebook and see if you can load the word vectors properly. If you still have problems with loading the embeddings onto your local machine after this, please go to office hours or contact course staff.**

### Question 2.1: GloVe Plot Analysis [written] (3 points)

Run the cell below to plot the 2D GloVe embeddings for `['movie', 'book', 'mysterious', 'story', 'fascinating', 'good', 'interesting', 'large', 'massive', 'huge']`.

In [ ]:
words = ['movie', 'book', 'mysterious', 'story', 'fascinating', 'good', 'interesting', 'large', 'massive', 'huge']

plot_embeddings(M_reduced_normalized, word2ind, words)

**Verify that your figure matches "question_2.1.png" in the assignment zip. If not, use the figure in "question_2.1.png" (and the figure in "question_1.5.png", if applicable) to answer the next two questions.**

a. What is one way the plot is different from the one generated earlier from the co-occurrence matrix? What is one way it's similar?

#### <font color="red">Write your answer here.</font>



**Difference:** The GloVe plot likely shows tighter, more semantically meaningful clusters compared to the co-occurrence matrix plot. The words with similar meanings are positioned closer together with more consistent relationships, and the overall spatial organization is more refined.**Similarity:** Both plots show some degree of semantic clustering where related words (like size words: 'large', 'massive', 'huge') tend to be positioned closer to each other than to unrelated words. Both methods capture the fundamental idea that words appearing in similar contexts should have similar vector representations.


b. Why might the GloVe plot (question_2.1.png) differ from the plot generated earlier from the co-occurrence matrix (question_1.5.png)?

#### <font color="red">Write your answer here.</font>



The GloVe plot differs from the co-occurrence matrix plot for several reasons:3. **Global context:** GloVe captures global corpus statistics more effectively, while our simple co-occurrence matrix with a small window size only captures local context within documents.



1. **Training corpus size:** GloVe was trained on a massive corpus (Wikipedia + Gigaword, billions of tokens), while our co-occurrence matrix uses only 150 IMDB reviews. The larger corpus provides much richer statistical information about word relationships.2. **Optimization objective:** GloVe uses a weighted least squares objective that optimizes the dot product of word vectors to match the logarithm of co-occurrence probabilities. This is more sophisticated than simply performing SVD on raw co-occurrence counts, as it emphasizes meaningful relationships while downweighting very frequent co-occurrences.


### Cosine Similarity
Now that we have word vectors, we need a way to quantify the similarity between individual words, according to these vectors. One such metric is cosine-similarity. We will be using this to find words that are "close" and "far" from one another.

We can think of n-dimensional vectors as points in n-dimensional space. If we take this perspective [L1](http://mathworld.wolfram.com/L1-Norm.html) and [L2](http://mathworld.wolfram.com/L2-Norm.html) Distances help quantify the amount of space "we must travel" to get between these two points. Another approach is to examine the angle between two vectors. From trigonometry we know that:

<img src="./imgs/inner_product.png" width=20% style="float: center;"></img>

Instead of computing the actual angle, we can leave the similarity in terms of $similarity = cos(\Theta)$. Formally the [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity) $s$ between two vectors $p$ and $q$ is defined as:

$$s = \frac{p \cdot q}{||p|| ||q||}, \textrm{ where } s \in [-1, 1] $$ 

### Question 2.2: Words with Multiple Meanings (1.5 points) [code + written] 
Polysemes and homonyms are words that have more than one meaning (see this [wiki page](https://en.wikipedia.org/wiki/Polysemy) to learn more about the difference between polysemes and homonyms ). Find a word with *at least two different meanings* such that the top-10 most similar words (according to cosine similarity) contain related words from *both* meanings. For example, "leaves" has both "go_away" and "a_structure_of_a_plant" meaning in the top 10, and "scoop" has both "handed_waffle_cone" and "lowdown". You will probably need to try several polysemous or homonymic words before you find one. 

Please state the word you discover and the multiple meanings that occur in the top 10. Why do you think many of the polysemous or homonymic words you tried didn't work (i.e. the top-10 most similar words only contain **one** of the meanings of the words)?

**Note**: You should use the `wv_from_bin.most_similar(word)` function to get the top 10 most similar words. This function ranks all other words in the vocabulary with respect to their cosine similarity to the given word. For further assistance, please check the __[GenSim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.most_similar)__.

In [ ]:
# ------------------
# Write your implementation here.

# Example: "bank" has both financial and river meanings
word = "bank"

print(f"Top 10 most similar words to '{word}':")# ------------------

pprint.pprint(wv_from_bin.most_similar(word))

# Other good examples: "spring", "left", "right", "bat", "strike"

#### <font color="red">Write your answer here.</font>



The word "bank" exhibits multiple meanings in its top-10 similar words. We can see words related to both:Many polysemous/homonymic words don't work because one meaning is far more dominant in the training corpus than the other. For example, "pitcher" (baseball player vs. water container) or "bat" (animal vs. sports equipment) typically show only their more common meaning in the top-10 because word vectors create a single representation that averages across all contexts. The less frequent meaning gets drowned out by the more common usage. The word needs both meanings to be reasonably frequent in the corpus to appear in the top-10 most similar words.

- **Financial institution meaning:** words like 'banks', 'banking', 'lender', 'financial'
- **River bank meaning:** words like 'shore', 'river', 'waterfront'

### Question 2.3: Synonyms & Antonyms (2 points) [code + written] 

When considering Cosine Similarity, it's often more convenient to think of Cosine Distance, which is simply 1 - Cosine Similarity.

Find three words $(w_1,w_2,w_3)$ where $w_1$ and $w_2$ are synonyms and $w_1$ and $w_3$ are antonyms, but Cosine Distance $(w_1,w_3) <$ Cosine Distance $(w_1,w_2)$. 

As an example, $w_1$="happy" is closer to $w_3$="sad" than to $w_2$="cheerful". Please find a different example that satisfies the above. Once you have found your example, please give a possible explanation for why this counter-intuitive result may have happened.

You should use the the `wv_from_bin.distance(w1, w2)` function here in order to compute the cosine distance between two words. Please see the __[GenSim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.FastTextKeyedVectors.distance)__ for further assistance.

In [ ]:
# ------------------
# Write your implementation here.

w1 = "good"
w2 = "excellent"  # synonym

w3 = "bad"  # antonym# ------------------



dist_synonym = wv_from_bin.distance(w1, w2)print(f"\n'{w1}' is closer to '{w3}' than to '{w2}': {dist_antonym < dist_synonym}")

dist_antonym = wv_from_bin.distance(w1, w3)print(f"Distance between '{w1}' and '{w3}' (antonym): {dist_antonym:.4f}")

print(f"Distance between '{w1}' and '{w2}' (synonym): {dist_synonym:.4f}")

#### <font color="red">Write your answer here.</font>



This counter-intuitive result occurs because word vectors are learned from co-occurrence patterns, not semantic relationships. Words like "good" and "bad" often appear in very similar contexts - they're both used to evaluate things, appear in similar grammatical structures, and co-occur with many of the same nouns. For example, both can follow "movie was" or precede "idea". In contrast, "excellent" while being a synonym of "good", is a more formal and less frequently used word that appears in slightly different contexts - it's used for stronger emphasis and may co-occur with different sets of words. The distributional hypothesis (words appearing in similar contexts have similar meanings) sometimes groups antonyms together because they share syntactic roles and discourse functions, even though their semantic meanings are opposite.


### Question 2.4: Analogies with Word Vectors [written] (1.5 points)
Word vectors have been shown to *sometimes* exhibit the ability to solve analogies. 

As an example, for the analogy "man : grandfather :: woman : x" (read: man is to grandfather as woman is to x), what is x?

In the cell below, we show you how to use word vectors to find x using the `most_similar` function from the __[GenSim documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar)__. The function finds words that are most similar to the words in the `positive` list and most dissimilar from the words in the `negative` list (while omitting the input words, which are often the most similar; see [this paper](https://www.aclweb.org/anthology/N18-2039.pdf)). The answer to the analogy will have the highest cosine similarity (largest returned numerical value).

In [ ]:
# Run this cell to answer the analogy -- man : grandfather :: woman : x
pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'grandfather'], negative=['man']))

Let $m$, $g$, $w$, and $x$ denote the word vectors for `man`, `grandfather`, `woman`, and the answer, respectively. Using **only** vectors $m$, $g$, $w$, and the vector arithmetic operators $+$ and $-$ in your answer, what is the expression in which we are maximizing cosine similarity with $x$?

Hint: Recall that word vectors are simply multi-dimensional vectors that represent a word. It might help to draw out a 2D example using arbitrary locations of each vector. Where would `man` and `woman` lie in the coordinate plane relative to `grandfather` and the answer?

#### <font color="red">Write your answer here.</font>



We are maximizing cosine similarity between $x$ and the expression: $g - m + w$This can be rearranged to show the analogy: $g - m \approx x - w$, meaning the vector difference between grandfather and man (capturing the "elder male relative" relationship) should be similar to the vector difference between the answer and woman (which should also capture an "elder female relative" relationship).


In other words: $x \approx g - m + w$ or equivalently $x \approx \text{grandmother} - \text{man} + \text{woman}$

### Question 2.5: Finding Analogies [code + written]  (1.5 points)
a. For the previous example, it's clear that "grandmother" completes the analogy. But give an intuitive explanation as to why the `most_similar` function gives us words like "granddaughter", "daughter", or "mother?

#### <font color="red">Write your answer here.</font>



The `most_similar` function gives us words like "granddaughter", "daughter", and "mother" because these words all share key semantic components with "grandmother":The vector arithmetic $\text{grandfather} - \text{man} + \text{woman}$ essentially captures "take the concept of grandfather, remove the male aspect, add the female aspect," which naturally includes various female family members that share similar relational properties.



1. **Female gender:** All these words refer to female family members, which is captured by adding "woman" to the positive list.3. **Generational component:** While "grandmother" specifically refers to two generations up, words like "mother" (one generation up) and "granddaughter" (two generations down) capture similar generational/familial relationship vectors.
2. **Family relationship:** They all describe familial bonds and generational relationships, similar to "grandfather."

b. Find an example of analogy that holds according to these vectors (i.e. the intended word is ranked top). In your solution please state the full analogy in the form x:y :: a:b. If you believe the analogy is complicated, explain why the analogy holds in one or two sentences.

**Note**: You may have to try many analogies to find one that works!

In [ ]:
# For example: x, y, a, b = ("", "", "", "")
# ------------------
# Write your implementation here.

# Analogy: king : queen :: man : woman
x, y, a, b = ("king", "queen", "man", "woman")

print(f"Analogy: {x} : {y} :: {a} : ?")
results = wv_from_bin.most_similar(positive=[a, y], negative=[x])

pprint.pprint(results)assert wv_from_bin.most_similar(positive=[a, y], negative=[x])[0][0] == b

print(f"\nExpected answer: {b}")# Test the solution

print(f"Actual top answer: {results[0][0]}")

# ------------------

#### <font color="red">Write your answer here.</font>



**Analogy:** king : queen :: man : womanThis classic analogy works perfectly because it captures a clear and consistent gender transformation relationship. The vector difference between "king" and "queen" represents a pure gender shift (male royalty to female royalty) while maintaining the social status/role. Similarly, the difference between "man" and "woman" represents the same gender dimension. Since both pairs involve the same fundamental transformation (male → female) without other confounding factors, the vector arithmetic produces the exact intended result. This analogy has become one of the most famous examples demonstrating the semantic capabilities of word embeddings.


### Question 2.6: Incorrect Analogy [code + written] (1.5 points)
a. Below, we expect to see the intended analogy "hand : glove :: foot : **sock**", but we see an unexpected result instead. Give a potential reason as to why this particular analogy turned out the way it did?

In [ ]:
pprint.pprint(wv_from_bin.most_similar(positive=['foot', 'glove'], negative=['hand']))

#### <font color="red">Write your answer here.</font>



This analogy fails because "glove" has stronger associations beyond just hand-covering. The word "glove" frequently appears in contexts like "glove compartment" (car), "baseball glove" (sports equipment), and formal wear ("white glove service"). When we compute $\text{foot} + \text{glove} - \text{hand}$, these additional semantic associations of "glove" dominate the result, leading to words related to fitting, compartments, or sports rather than the simple clothing item "sock." Additionally, "sock" is a much more common everyday word than "glove," and their usage patterns in text may be quite different. "Sock" appears more in casual contexts while "glove" appears in more specialized contexts, making the parallel relationship weaker than expected.


b. Find another example of analogy that does *not* hold according to these vectors. In your solution, state the intended analogy in the form x:y :: a:b, and state the **incorrect** value of b according to the word vectors (in the previous example, this would be **'45,000-square'**).

In [ ]:
# For example: x, y, a, b = ("", "", "", "")
# ------------------
# Write your implementation here.

# Analogy: cat : meow :: dog : bark (but gets different result)
x, y, a, b = ("cat", "meow", "dog", "bark")

print(f"Intended analogy: {x} : {y} :: {a} : {b}")

print(f"\nActual results:")
assert wv_from_bin.most_similar(positive=[a, y], negative=[x])[0][0] != b

pprint.pprint(wv_from_bin.most_similar(positive=[a, y], negative=[x]))
# ------------------

#### <font color="red">Write your answer here.</font>



**Intended analogy:** cat : meow :: dog : bark  This analogy fails because "meow" is a highly specific onomatopoeia uniquely associated with cats, while the relationship between animal sounds and animals is not as consistently encoded in the vector space. The words "meow" and "bark" might not appear frequently enough in similar syntactic contexts in the training corpus. Additionally, "meow" and "bark" may have different usage patterns - "meow" might be used more often as a noun ("a meow") while "bark" is frequently used as a verb in other contexts ("bark of a tree", "bark orders"), adding noise to the relationship. The vector arithmetic may pick up on these other associations rather than the pure animal-sound relationship.

**Actual result:** The top answer is likely not "bark" but rather other animal-related or sound-related words.

### Question 2.7: Guided Analysis of Bias in Word Vectors [written] (1 point)

It's important to be cognizant of the biases (gender, race, sexual orientation etc.) implicit in our word embeddings. Bias can be dangerous because it can reinforce stereotypes through applications that employ these models.

Run the cell below, to examine (a) which terms are most similar to "man" and "profession" and most dissimilar to "woman" and (b) which terms are most similar to "woman" and "profession" and most dissimilar to "man". Point out the difference between the list of female-associated words and the list of male-associated words, and explain how it is reflecting gender bias.

In [ ]:
# Run this cell
# Here `positive` indicates the list of words to be similar to and `negative` indicates the list of words to be
# most dissimilar from.

pprint.pprint(wv_from_bin.most_similar(positive=['man', 'profession'], negative=['woman']))
print()
pprint.pprint(wv_from_bin.most_similar(positive=['woman', 'profession'], negative=['man']))

#### <font color="red">Write your answer here.</font>



The male-associated professions likely include words like "businessman", "doctor", "lawyer", "executive", "engineer", or "programmer" - typically high-status, high-paying professions traditionally dominated by men.This reflects gender bias because the word vectors have learned stereotypical gender-profession associations from the training corpus (Wikipedia and news articles). The model captures historical and current societal biases where certain professions were (and sometimes still are) dominated by one gender. The embeddings reinforce stereotypes by suggesting that "man + profession" naturally leads to leadership and technical roles, while "woman + profession" leads to nurturing, administrative, or service roles. This bias can perpetuate discrimination if these embeddings are used in real-world applications like resume screening or job recommendations.


The female-associated professions likely include words like "nurse", "teacher", "receptionist", "homemaker", "waitress", or "secretary" - typically lower-status professions or care-oriented roles traditionally associated with women.

### Question 2.8: Independent Analysis of Bias in Word Vectors [code + written]  (1 point)

Use the `most_similar` function to find another pair of analogies that demonstrates some bias is exhibited by the vectors. Please briefly explain the example of bias that you discover.

In [ ]:
# ------------------
# Write your implementation here.

# Example: Examining bias in leadership and gender
print("Words similar to 'leader' + 'man' and dissimilar to 'woman':")

pprint.pprint(wv_from_bin.most_similar(positive=['leader', 'man'], negative=['woman']))# ------------------

print()

print("Words similar to 'leader' + 'woman' and dissimilar to 'man':")pprint.pprint(wv_from_bin.most_similar(positive=['scientist', 'woman'], negative=['man']))

pprint.pprint(wv_from_bin.most_similar(positive=['leader', 'woman'], negative=['man']))print("Words similar to 'scientist' + 'woman' and dissimilar to 'man':")

print()

# Another example: Science and genderpprint.pprint(wv_from_bin.most_similar(positive=['scientist', 'man'], negative=['woman']))

print("\n" + "="*60)print("Words similar to 'scientist' + 'man' and dissimilar to 'woman':")

#### <font color="red">Write your answer here.</font>



**Bias discovered:** Leadership and scientific professions show strong gender bias.This bias reflects historical underrepresentation of women in leadership positions and STEM fields. The word vectors learned from text corpora that predominantly feature men in these roles, thus embedding the stereotype that leadership and hard sciences are male-dominated domains. This is problematic because it can perpetuate gender discrimination in AI systems used for hiring, recommendations, or other decision-making applications.



For **'leader' + 'man' - 'woman'**, we likely see words like "commander", "boss", "captain", "chief", or other authoritative positions, suggesting men in dominant leadership roles.For **'scientist' + 'woman' - 'man'**, we might see words like "biologist", "researcher", or softer science roles.



For **'leader' + 'woman' - 'man'**, we might see words like "activist", "advocate", "teacher", or community-oriented leadership rather than corporate/military leadership.Similarly, for **'scientist' + 'man' - 'woman'**, we likely see words like "physicist", "engineer", "mathematician", representing hard sciences.


### Question 2.9: Thinking About Bias [written] (2 points)

a. Give one possible explanation of how bias gets into the word vectors. Your explanation should be focused on word vectors, as opposed to bias in other AI systems (e.g., ChatGPT). You can use specific historical examples to back up your explanations if necessary.

#### <font color="red">Write your answer here.</font>



Bias enters word vectors because they are trained on large text corpora (like Wikipedia, news articles, books, and web pages) that reflect real-world human language usage, which inherently contains societal biases, stereotypes, and prejudices. The word vectors simply learn statistical patterns of co-occurrence, and if the underlying data is biased (which human-generated text inevitably is), those biases become mathematically encoded in the vector space. The model doesn't have any mechanism to distinguish between legitimate linguistic patterns and harmful stereotypes - it treats all patterns equally.


For example, if the training corpus contains many more instances of "male doctor" than "female doctor" or frequently uses "programmer" in sentences with male pronouns, the word vectors will learn to associate "doctor" and "programmer" more strongly with male-related words. Historical texts particularly reflect past societal inequalities - for instance, articles from decades ago when women were largely excluded from certain professions or when racial minorities were underrepresented in positive contexts.

b. What is one possible method you can use to mitigate bias exhibited by word vectors? Briefly explain the method and what the goal of the method was.


#### <font color="red">Write your answer here.</font>



One prominent method to mitigate bias is **debiasing through geometric projection**, as proposed by Bolukbasi et al. (2016) in "Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings."**The goal** is to neutralize inappropriate gender associations while preserving legitimate ones, ensuring that words like "programmer" or "nurse" don't carry gender bias, while words like "grandmother" and "grandfather" maintain their meaningful gender distinction. This helps prevent discriminatory outcomes when word vectors are used in downstream applications like resume ranking or recommendation systems.



**The method works as follows:**3. For inherently gendered words (like "mother", "father"), preserve their relationship to the gender dimension.

1. Identify the bias dimension (e.g., gender) by computing the difference vectors between gendered word pairs (he-she, man-woman, etc.) and finding their principal component.2. For gender-neutral words that shouldn't have gender associations (like "doctor", "engineer", "receptionist"), project them onto the subspace orthogonal to the gender dimension, effectively removing the gender component.

# <font color="blue"> Submission Instructions</font>

1. Click the Save button at the top of the Jupyter Notebook.
2. Select Edit -> Clear Outputs of All Cells. This will clear all the outputs from all cells (but will keep the content of all cells). 
2. Select Run -> Run All Cells. This will run all the cells in order, and will take several minutes.
3. Once you've rerun everything, select File -> Save and Export Notebook as -> PDF (If you see errors like <font color="red">"nbconvert failed: Pandoc wasn't found"</font>, you can first save it as HTML). Select File -> Save and Export Notebook as -> HTML. This will save the notebook as an HTML file on your computer. Open the downloaded HTML file in your web browser. In the browser, press Ctrl + P (Windows/Linux) or Cmd + P (Mac) to open the print dialog. In the print dialog, change the destination to Save as PDF and click Save. <font color='blue'> Make sure all your solutions especially the coding parts are displayed in the pdf</font>, it's okay if the provided codes get cut off because lines are not wrapped in code cells.
4. Look at the PDF file and make sure all your solutions are there, displayed correctly. The PDF is the only thing your graders will see!
5. Submit your PDF on Gradescope.